<a href="https://colab.research.google.com/github/skywalker0803r/base_ball_detect_lab/blob/main/%E6%A3%92%E7%90%83%E8%BB%8C%E8%B7%A1%E5%81%B5%E6%B8%AC%E5%AF%A6%E9%A9%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install baseballcv ultralytics

In [2]:
from baseballcv.functions import LoadTools
from ultralytics import YOLO

# 以偵測棒球為例
load_tools = LoadTools()
model_path = load_tools.load_model("ball_tracking")  # 會自動下載模型並返回路徑

model = YOLO(model_path)
results = model.predict("/content/彩色中央區域.png", imgsz=640,device='cuda:0')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
2025-06-17 05:57:28,184 - LoadTools - INFO - Model downloaded to models/od/YOLO/ball_tracking/model_weights/ball_tracking.pt


INFO:LoadTools:Model downloaded to models/od/YOLO/ball_tracking/model_weights/ball_tracking.pt



image 1/1 /content/彩色中央區域.png: 576x640 2 baseballs, 82.5ms
Speed: 18.9ms preprocess, 82.5ms inference, 345.8ms postprocess per image at shape (1, 3, 576, 640)


In [ ]:
import cv2
import matplotlib.pyplot as plt
def draw_ultralytics_results(results_list, conf_threshold=0.5):
    # 預設只畫第一張
    results = results_list[0]

    img = results.orig_img.copy()  # 取得原始圖片 numpy 陣列
    names = results.names  # 類別名稱字典 {id: label}
    boxes = results.boxes.xyxy.cpu().numpy()  # 預測框座標 (N,4) [x1,y1,x2,y2]
    scores = results.boxes.conf.cpu().numpy()  # 信心度 (N,)
    class_ids = results.boxes.cls.cpu().numpy().astype(int)  # 類別ID (N,)

    for box, score, cls_id in zip(boxes, scores, class_ids):
        if score < conf_threshold:
            continue  # 忽略信心度低的

        x1, y1, x2, y2 = box.astype(int)
        label = names.get(cls_id, str(cls_id))
        text = f"{label} {score:.2f}"

        # 畫邊框 (綠色)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # 畫標籤文字 (藍色)
        cv2.putText(img, text, (x1, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # 用 matplotlib 顯示 (RGB 格式)
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"YOLOv8 Detection Result (conf > {conf_threshold})")
    plt.show()

# 呼叫時可用預設 0.5 門檻
draw_ultralytics_results(results)

In [6]:
#/content/pitch_0001.mp4
#幫我寫一個函數 該函數基於前面實驗成功可以偵測單張棒球位置的 draw_ultralytics_results
#我希望輸入這個影片他會輸出完整的棒球路徑軌跡

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def extract_baseball_trajectory_from_video(video_path, model, conf_threshold=0.5, baseball_class_name="baseball"):
    """
    從影片逐幀偵測棒球位置，回傳軌跡座標列表，並畫出軌跡圖。

    參數：
        video_path: str，影片檔案路徑
        model: 已載入的 YOLOv8 模型物件
        conf_threshold: float，信心度門檻，低於此忽略
        baseball_class_name: str，棒球類別名稱（依你模型的類別而定）

    回傳：
        trajectory: list of (frame_index, x_center, y_center)
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"無法開啟影片檔：{video_path}")

    trajectory = []
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # 讀完了

        # YOLOv8 預測，model 物件直接丟 np.ndarray 也行
        results = model(frame,verbose=False)  # 取得 list[Results]

        # 因為只處理一張圖，所以取 results[0]
        res = results[0]

        # 取得類別名稱與boxes/conf
        names = res.names
        boxes = res.boxes.xyxy.cpu().numpy()
        scores = res.boxes.conf.cpu().numpy()
        class_ids = res.boxes.cls.cpu().numpy().astype(int)

        # 找出棒球位置（第一個符合信心度且是棒球的框）
        ball_found = False
        for box, score, cls_id in zip(boxes, scores, class_ids):
            if score < conf_threshold:
                continue
            label = names.get(cls_id, str(cls_id))
            if label == baseball_class_name:
                x1, y1, x2, y2 = box.astype(int)  # <-- 修正這行
                x_center = (x1 + x2) / 2
                y_center = (y1 + y2) / 2
                trajectory.append((frame_idx, x_center, y_center))
                ball_found = True
                print(frame_idx,'找到棒球')#同時把畫面畫出來
                # 畫框與標籤
                text = f"{label} {score:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, text, (x1, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                break

        # 如果該幀沒找到棒球，也可以記錄 None 或忽略
        if not ball_found:
            trajectory.append((frame_idx, None, None))

        # 顯示圖片（用 matplotlib）
        if ball_found:
          frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          plt.figure(figsize=(8, 6))
          plt.imshow(frame_rgb)
          plt.axis('off')
          plt.title(f"Frame {frame_idx}")
          plt.show()

        # 準備進下一個迴圈
        frame_idx += 1

    cap.release()
    # 畫出軌跡（濾掉 None）
    xs = [p[1] for p in trajectory if p[1] is not None]
    ys = [p[2] for p in trajectory if p[2] is not None]

    plt.figure(figsize=(12, 8))
    plt.title("Baseball Trajectory")
    plt.xlabel("X (pixels)")
    plt.ylabel("Y (pixels)")
    plt.gca().invert_yaxis()  # 影像座標Y軸向下，畫圖翻轉讓Y軸向上

    plt.plot(xs, ys, marker='o', linestyle='-', color='r')
    plt.scatter(xs, ys, color='b', s=10)
    plt.show()

    return trajectory
# 測試
trajectory = extract_baseball_trajectory_from_video(video_path='/content/pitch_0001.mp4', model=model, conf_threshold=0.5, baseball_class_name="baseball")

In [ ]:
trajectory

In [ ]:

def extract_longest_valid_segment(trajectory):
    max_len = 0
    current = []
    best = []

    for item in trajectory:
        frame_idx, x, y = item
        if x is not None and y is not None:
            current.append(item)
        else:
            if len(current) > max_len:
                max_len = len(current)
                best = current
            current = []
    # 最後一段也可能是最長的
    if len(current) > max_len:
        best = current

    return best
# 假設你已經有 trajectory
longest_segment = extract_longest_valid_segment(trajectory)

# 印出結果
for frame_idx, x, y in longest_segment:
    print(f"Frame {frame_idx}: x={x}, y={y}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def animate_trajectory(segment):
    xs = [x for _, x, _ in segment]
    ys = [y for _, _, y in segment]

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_xlim(min(xs) - 10, max(xs) + 10)
    ax.set_ylim(max(ys) + 10, min(ys) - 10)
    ax.set_title("Baseball Trajectory Animation")
    ax.set_xlabel("X (pixels)")
    ax.set_ylabel("Y (pixels)")
    ax.grid(True)

    scat, = ax.plot([], [], 'ro', markersize=5)
    line, = ax.plot([], [], 'b-', linewidth=2)

    def init():
        scat.set_data([], [])
        line.set_data([], [])
        return scat, line

    def update(frame):
        x_data = xs[:frame+1]
        y_data = ys[:frame+1]
        scat.set_data([x_data[-1]], [y_data[-1]])  # ✅ 修正為列表
        line.set_data(x_data, y_data)
        return scat, line

    ani = animation.FuncAnimation(
        fig, update, frames=len(xs),
        init_func=init, blit=True, interval=200, repeat=False
    )

    plt.close(fig)  # 防止重複顯示空圖
    return HTML(ani.to_jshtml())  # ✅ Colab 上播放動畫

# 顯示動畫
animate_trajectory(longest_segment)


In [28]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import Image
import os

def animate_trajectory_and_save(segment, save_path='trajectory.gif'):
    xs = [x for _, x, _ in segment]
    ys = [y for _, _, y in segment]

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_xlim(min(xs) - 10, max(xs) + 10)
    ax.set_ylim(max(ys) + 10, min(ys) - 10)
    ax.set_title("Baseball Trajectory Animation")
    ax.set_xlabel("X (pixels)")
    ax.set_ylabel("Y (pixels)")
    ax.grid(True)

    scat, = ax.plot([], [], 'ro', markersize=5)
    line, = ax.plot([], [], 'b-', linewidth=2)

    def init():
        scat.set_data([], [])
        line.set_data([], [])
        return scat, line

    def update(frame):
        x_data = xs[:frame+1]
        y_data = ys[:frame+1]
        scat.set_data([x_data[-1]], [y_data[-1]])
        line.set_data(x_data, y_data)
        return scat, line

    ani = animation.FuncAnimation(
        fig, update, frames=len(xs),
        init_func=init, blit=True, interval=200, repeat=False
    )

    # 儲存為 .gif 使用 Pillow writer
    ani.save(save_path, writer='pillow', fps=5)  # 每秒5幀
    plt.close(fig)

    print(f"動畫已儲存為 {save_path}")
    return save_path


In [ ]:
# 儲存動畫並顯示結果
gif_path = animate_trajectory_and_save(longest_segment, save_path='baseball_trajectory.gif')
Image(filename=gif_path)  # 在 Jupyter 或 Colab 中顯示 .gif
